In [1]:
import pandas as pd
import numpy as np
path = '/Users/gunnvantsaini/Documents/Data/Kaggle/StackoverflowTags/stack-overflow-data.csv'

In [2]:
data = pd.read_csv(path)

In [3]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [5]:
print(data['post'].iloc[0])

what is causing this behavior  in our c# datetime type  <pre><code>[test] public void sadness() {    var datetime = datetime.utcnow;    assert.that(datetime  is.equalto(datetime.parse(datetime.tostring()))); } </code></pre>   failed :   <pre><code> expected: 2011-10-31 06:12:44.000  but was:  2011-10-31 06:12:44.350 </code></pre>   i wish to know what is happening behind the scenes in tostring() etc to cause this behavior.    edit after seeing jon s answer :   <pre><code>[test] public void newsadness() {     var datetime = datetime.utcnow;     assert.that(datetime  is.equalto(datetime.parse(datetime.tostring( o )))); } </code></pre>   result :   <pre><code>expected: 2011-10-31 12:03:04.161 but was:  2011-10-31 06:33:04.161 </code></pre>   same result with capital and small  o  . i m reading up the docs  but still unclear.


In [6]:
print(data['post'].iloc[30])

how to chain expressions inside ngclass when using the {...}[] form  how can i add another expression to an <code>ng-class</code> directive that uses this form:   <pre><code>ng-class= {true: loading   false: loading-done }[data.loader===null]  </code></pre>   i d like to add something like this to the list:   <pre><code>{highlight:isspecial} </code></pre>   is it possible without expanding the first expression     thanks.


In [7]:
data['tags'].unique()

array(['c#', 'asp.net', 'objective-c', '.net', 'python', 'angularjs',
       'iphone', 'ruby-on-rails', 'ios', 'c', 'sql', 'java', 'jquery',
       'css', 'c++', 'php', 'android', 'mysql', 'javascript', 'html'],
      dtype=object)

In [8]:
data.shape

(40000, 2)

In [10]:
data['tags'].value_counts(normalize=True)

c#               0.05
asp.net          0.05
javascript       0.05
mysql            0.05
android          0.05
php              0.05
c++              0.05
css              0.05
jquery           0.05
java             0.05
sql              0.05
c                0.05
ios              0.05
ruby-on-rails    0.05
iphone           0.05
angularjs        0.05
python           0.05
.net             0.05
objective-c      0.05
html             0.05
Name: tags, dtype: float64

In [11]:
### split data into train-test
train = data.sample(frac=0.80,random_state=100)

In [13]:
test = data.drop(train.index,axis=0) 

In [15]:
train.index

Int64Index([37682, 31007, 13192,  2138, 13481, 19133, 14338, 16957, 26277,
            10773,
            ...
            27337,  7964, 39698, 11262, 26962,  2346, 12711, 39512, 13902,
            34151],
           dtype='int64', length=32000)

In [16]:
train['tags'].value_counts(normalize=True)

ruby-on-rails    0.050906
sql              0.050688
c#               0.050656
jquery           0.050563
c++              0.050438
objective-c      0.050375
javascript       0.050344
python           0.050313
c                0.050281
.net             0.050000
php              0.049969
asp.net          0.049844
angularjs        0.049781
java             0.049750
css              0.049750
mysql            0.049719
iphone           0.049594
ios              0.049531
html             0.049406
android          0.048094
Name: tags, dtype: float64

In [17]:
from sklearn.model_selection import train_test_split

In [19]:
### Create a new dataset where startification on classes is ensured; we will compare the model performace
X,y = data['post'].values,data['tags'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,stratify=y)

In [23]:
####### Data Preparation ########
### Build 4 tfidf matrices:
# X_train (fit_transform())
# X_test (transform())
# Train (fit_transform)
# Test (transform)
from sklearn.feature_extraction import text

In [26]:
tfidf_df = text.TfidfVectorizer(input = train['post'].tolist(),stop_words='english',max_features=4000)
matrix_train_df = tfidf_df.fit_transform(train['post'].tolist())
matrix_train_df.shape

(32000, 4000)

In [27]:
matrix_test_df = tfidf_df.transform(test['post'].tolist())

In [28]:
matrix_test_df.shape

(8000, 4000)

In [29]:
tfidf_np = text.TfidfVectorizer(input = X_train.tolist(),stop_words='english',max_features=4000)
matrix_train_np = tfidf_np.fit_transform(X_train.tolist())
matrix_test_np = tfidf_np.transform(X_test.tolist())

In [31]:
from sklearn.preprocessing import LabelEncoder

In [33]:
enc_df = LabelEncoder()
y_train_df = enc_df.fit_transform(train['tags'])
y_test_df = enc_df.transform(test['tags'])

In [34]:
enc_np = LabelEncoder()
y_train_np = enc_np.fit_transform(y_train)
y_test_np = enc_np.transform(y_test)

In [35]:
len(train['tags'].unique())

20

In [36]:
matrix_test_np.shape

(8000, 4000)

In [37]:
matrix_train_df.shape

(32000, 4000)

In [39]:
##### Neural Network Model #####

In [38]:
import tensorflow as tf

In [41]:
## Model 1
inputs =tf.keras.Input(shape=(4000,))  ##4000 columns in X
h1 = tf.keras.layers.Dense(units=128,activation='relu')(inputs)   ## 128 neurons in hidden layer with 784 inputs to each of the neurons
h2 = tf.keras.layers.Dense(units=64,activation='relu')(h1)
h3 = tf.keras.layers.Dense(units=32,activation='relu')(h2)
probs = tf.keras.layers.Dense(units=20,activation='softmax')(h3)  ##20 output neurons for predicted probability of 20 classes
model1 = tf.keras.Model(inputs,probs)

In [42]:
## Model 2
inputs =tf.keras.Input(shape=(4000,))  ##4000 columns in X
h1 = tf.keras.layers.Dense(units=128,activation='relu')(inputs)   ## 128 neurons in hidden layer with 784 inputs to each of the neurons
h2 = tf.keras.layers.Dense(units=64,activation='relu')(h1)
h3 = tf.keras.layers.Dense(units=32,activation='relu')(h2)
probs = tf.keras.layers.Dense(units=20,activation='softmax')(h3)  ##20 output neurons for predicted probability of 20 classes
model2 = tf.keras.Model(inputs,probs)

In [43]:
## Compile Models
model1.compile(loss = 'sparse_categorical_crossentropy',metrics = ['acc'])
model2.compile(loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [45]:
### Fit model 1 on df data (Non Stratified Data)
model1.fit(matrix_train_df.toarray(),y_train_df,epochs=10,batch_size=32,validation_split=0.10)

2022-01-18 11:21:04.038821: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-18 11:21:04.220770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
892/900 [============================>.] - ETA: 0s - loss: 1.1808 - acc: 0.6671

2022-01-18 11:21:09.376608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


900/900 [==============================] - 6s 6ms/step - loss: 1.1763 - acc: 0.6680 - val_loss: 0.6752 - val_acc: 0.7816
Epoch 2/10
900/900 [==============================] - 5s 5ms/step - loss: 0.5790 - acc: 0.8169 - val_loss: 0.6180 - val_acc: 0.8025
Epoch 3/10
900/900 [==============================] - 5s 5ms/step - loss: 0.4939 - acc: 0.8441 - val_loss: 0.6465 - val_acc: 0.7959
Epoch 4/10
900/900 [==============================] - 5s 6ms/step - loss: 0.4397 - acc: 0.8617 - val_loss: 0.6329 - val_acc: 0.8012
Epoch 5/10
900/900 [==============================] - 5s 6ms/step - loss: 0.3954 - acc: 0.8764 - val_loss: 0.6716 - val_acc: 0.8012
Epoch 6/10
900/900 [==============================] - 5s 5ms/step - loss: 0.3571 - acc: 0.8880 - val_loss: 0.6809 - val_acc: 0.8031
Epoch 7/10
900/900 [==============================] - 5s 5ms/step - loss: 0.3244 - acc: 0.8994 - val_loss: 0.7234 - val_acc: 0.7972
Epoch 8/10
900/900 [==============================] - 5s 6ms/step - loss: 0.2926 - acc:

In [46]:
### Fit model 2 on df data (Stratified Data)
model2.fit(matrix_train_np.toarray(),y_train_np,epochs=10,batch_size=32,validation_split=0.10)

Epoch 1/10
  8/900 [..............................] - ETA: 6s - loss: 2.9910 - acc: 0.0781  

2022-01-18 11:22:03.725371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


896/900 [============================>.] - ETA: 0s - loss: 1.1707 - acc: 0.6673

2022-01-18 11:22:08.832912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


900/900 [==============================] - 6s 6ms/step - loss: 1.1682 - acc: 0.6680 - val_loss: 0.7034 - val_acc: 0.7731
Epoch 2/10
900/900 [==============================] - 5s 6ms/step - loss: 0.5762 - acc: 0.8204 - val_loss: 0.6665 - val_acc: 0.7884
Epoch 3/10
900/900 [==============================] - 5s 6ms/step - loss: 0.4932 - acc: 0.8450 - val_loss: 0.6813 - val_acc: 0.7862
Epoch 4/10
900/900 [==============================] - 5s 6ms/step - loss: 0.4410 - acc: 0.8633 - val_loss: 0.6918 - val_acc: 0.7891
Epoch 5/10
900/900 [==============================] - 5s 5ms/step - loss: 0.4004 - acc: 0.8743 - val_loss: 0.7204 - val_acc: 0.7878
Epoch 6/10
900/900 [==============================] - 5s 5ms/step - loss: 0.3638 - acc: 0.8864 - val_loss: 0.7393 - val_acc: 0.7847
Epoch 7/10
900/900 [==============================] - 5s 5ms/step - loss: 0.3318 - acc: 0.8978 - val_loss: 0.7439 - val_acc: 0.7887
Epoch 8/10
900/900 [==============================] - 5s 5ms/step - loss: 0.3002 - acc:

In [50]:
### Accuracy check on test set ###
pred_non_stratified = model1.predict(matrix_test_df.toarray()).argmax(axis=1)

In [51]:
pred_stratified = model2.predict(matrix_test_np.toarray()).argmax(axis=1)

2022-01-18 11:29:25.170530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [53]:
(pred_non_stratified == y_test_df).mean()

0.78575

In [59]:
(y_test_np == pred_stratified).mean()

0.795

In [81]:
#### Use our model on real text ###
q1 = '''I am trying to run Django inside WordPress like WordPress at main url www.wptesting.com and Django at suburl www.wptesting.com/django . Django main root url Is working fine at www.wptesting.com/django but its suburl e.g., admin is not working as it should be www.wptesting.com/django/admin . However, whenever I tried to request admin url it goes converts into www.wptesting.comhttp%3a//wptesting.com/django/admin

I am running WordPress and Django with Apache and mod_wsgi , my conf file for apache is as follows :

<VirtualHost *:80>

WSGIScriptAlias /django /path_to_project/wsgi.py

ServerName wptesting.com
ServerAlias www.wptesting.com

DocumentRoot /var/www/html/wordpress

<Directory /var/www/html/wordpress/>
AllowOverride All
Order allow,deny

allow from all
#            Options Indexes FollowSymLinks
#            Require all granted
</Directory>


<Directory /path_to_project/>
            Options Indexes FollowSymLinks
            Require all granted
</Directory>


</VirtualHost>
I have asked one question earlier about configuring Django from subdirectory of WordPress with Apache and wsgi -> you can see the question here

Also If I tried to access any url which is not in Django project then it is giving the standard 404 not found error but when I try to access any valid url like admin it is giving the error mention above.

Edited : My Urls.py file :

from django.conf.urls import patterns, include, url
from django.contrib import admin
from django.views.generic import TemplateView

urlpatterns = patterns('',
    # Examples:
    # url(r'^$', 'dev_redis.views.home', name='home'),
    # url(r'^blog/', include('blog.urls')),

    url(r'^$',TemplateView.as_view(template_name='index.html')),
    url(r'^admin/', include(admin.site.urls)),

    url(r'^cache/', 'redis_app.views.redisTest'),

)'''

In [82]:
## Vectorize the text
vec1 = tfidf_df.transform([q1])
vec2 = tfidf_np.transform([q1])

In [96]:
model1.predict(vec1.toarray()).argmax()

17

In [97]:
model2.predict(vec2.toarray()).argmax()

17

In [98]:
enc_df.inverse_transform([17])

array(['python'], dtype=object)

In [99]:
enc_np.inverse_transform([17])

array(['python'], dtype=object)

In [100]:
path = '/Users/gunnvantsaini/Documents/Work/Jigsaw Academy/Course Review/Textmining_casestudy/products.csv'
data=pd.read_csv(path)
data.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [110]:
### find out top 5 most similar products to each product given in the data
tfidf = text.TfidfVectorizer(input = data['product_name'].tolist(),stop_words='english')
matrix = tfidf.fit_transform(data['product_name'].tolist())

In [111]:
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
sim = cosine_similarity(matrix,dense_output=False)

In [114]:
sim

<49688x49688 sparse matrix of type '<class 'numpy.float64'>'
	with 95510928 stored elements in Compressed Sparse Row format>

In [129]:
from tqdm import tqdm
sim_products = []
for row in tqdm(sim,total=sim.shape[0]):
    idx = row.toarray()[0].argsort()[-6:-1]
    prods = data.iloc[idx]['product_name'].tolist()
    prods = ",".join(prods)
    sim_products.append(prods)

100%|█████████████████████████████████| 49688/49688 [00:42<00:00, 1166.74it/s]


In [130]:
data['sim_products']=sim_products

In [132]:
data.head(5)

,product_id,product_name,aisle_id,department_id,sim_products
0,1,Chocolate Sandwich Cookies,61,19,"Chocolate Mint Creme Sandwich Cookies,Reduced ..."
1,2,All-Seasons Salt,104,13,"Salt,Salsa For All Seasons Medium Chipotle Sal..."
2,3,Robust Golden Unsweetened Oolong Tea,94,7,"Oolong Tea Classic,Organic Oolong Tea,Unsweete..."
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,"Smart Ones Spaghetti with Meat Sauce,Smart One..."
4,5,Green Chile Anytime Sauce,5,13,"Mild Enchilada Green Chile Sauce,Mild Green Ch..."


### NER

In [1]:
import spacy

In [2]:
nlp=spacy.load("en_core_web_sm") ### loaded the model

In [3]:
text = "Joe Biden visited China."

In [4]:
doc=nlp(text) ## obtained predictions from your model

In [6]:
for ent in doc.ents:
    print(ent.text,ent.label_)

Joe Biden PERSON
China GPE


In [7]:
with open("../codes/data/data/4625.txt",'r') as r:
    text = r.read()
print(text)
### remove the name of the preson with a string 'masked'

Karen,

Thank you for the update.  It looks like we'll plan on having the EBS/Avaya 
meetings on January 10th and 11th, 2001.  The first day will be a full day, 
the second will be 1/2 day, a.m. session.  You have asked me to provide a 
list of Enron attendees, titles, which day(s) they would likely attend, and 
some background information on the meeting(s) purposes.  An explanation of 
the meetings' proposed focus and probable attendees is in the attached 
meeting notes.  

The notes are from the November meeting which we coordinated and held for 
Enron Broadband Services and Dave Johnson.  By copy of this note to Kim 
Godfrey, we'll update the EBS executives on the meetings, and work on 
arranging their calendar availability.  So far, we have had the EBS execs' 
calendars penciled in for the time slot of January 9-11.  At this point, I 
would expect that the EBS attendee list would look something like this:  

Jim Crowder, VP, Enterprise Services; Enron Broadband Services - day 2 
Ev

In [13]:
doc = nlp(text)

In [14]:
mentions = []
for ent in doc.ents:
    if ent.label_=="PERSON":
        if ent.text not in mentions:
            mentions.append(ent.text)

In [15]:
mentions

['Karen',
 'Dave Johnson',
 'Kim \nGodfrey',
 'Jim Crowder',
 'Everett Plante',
 'Larry Ciscon',
 'Larry',
 'Steve Pearlman',
 'Kim Godfrey',
 'Jeff Youngflesh',
 'Serge \nMinassian',
 'John Stephenson',
 'Barbara Korp',
 'Serge Minassian',
 'Oswald',
 'Jeff',
 'Dave',
 'Karen Oswald']

In [16]:
for mention in mentions:
    text = text.replace(mention,"masked")
print(text)

masked,

Thank you for the update.  It looks like we'll plan on having the EBS/Avaya 
meetings on January 10th and 11th, 2001.  The first day will be a full day, 
the second will be 1/2 day, a.m. session.  You have asked me to provide a 
list of Enron attendees, titles, which day(s) they would likely attend, and 
some background information on the meeting(s) purposes.  An explanation of 
the meetings' proposed focus and probable attendees is in the attached 
meeting notes.  

The notes are from the November meeting which we coordinated and held for 
Enron Broadband Services and masked.  By copy of this note to masked, we'll update the EBS executives on the meetings, and work on 
arranging their calendar availability.  So far, we have had the EBS execs' 
calendars penciled in for the time slot of January 9-11.  At this point, I 
would expect that the EBS attendee list would look something like this:  

masked, VP, Enterprise Services; Enron Broadband Services - day 2 
masked, VP and CIO